In [ ]:
# 여러가지 시도(DI는 버리자)

In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import optuna

import os

# 작업 디렉토리를 변경
os.chdir('/Users/yudayeon/Desktop/LGAimers')

# 변경된 디렉토리 확인
print("🔥 변경된 작업 디렉토리:", os.getcwd())

# 이제 상대 경로 사용 가능
X_train_encoded = pd.read_csv('data/train_encoded.csv')
X_test_encoded = pd.read_csv('data/test_encoded.csv')

# 시술 유형 컬럼명 설정 (실제 컬럼명이 다를 수도 있으니 확인 필요!)
procedure_column = "시술 유형"

# 시술 유형에 포함된 고유 값 출력
print("🔥 시술 유형별 고유 값 🔥")
print(X_train_encoded[procedure_column].unique())

# 시술 유형별 개수 확인
print("\n🔥 시술 유형별 개수 🔥")
print(X_train_encoded[procedure_column].value_counts())

🔥 변경된 작업 디렉토리: /Users/yudayeon/Desktop/LGAimers
🔥 시술 유형별 고유 값 🔥
[1. 0.]

🔥 시술 유형별 개수 🔥
시술 유형
1.0    250060
0.0      6291
Name: count, dtype: int64


In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

# ✅ 타겟 변수와 특성 분리
y = X_train_encoded['임신 성공 여부']
X = X_train_encoded.drop('임신 성공 여부', axis=1)

# ✅ 데이터 분할 (훈련 / 검증)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ✅ 최적 하이퍼파라미터 적용한 모델 정의
rf_model = RandomForestClassifier(
    n_estimators=215,
    max_depth=7,
    min_samples_split=2,
    min_samples_leaf=8,
    max_features=None,
    random_state=42
)

xgb_model = xgb.XGBClassifier(
    n_estimators=359,
    max_depth=5,
    learning_rate=0.045093620098282834,
    subsample=0.6669970717173888,
    colsample_bytree=0.934119080218835,
    random_state=42
)

lgb_model = lgb.LGBMClassifier(
    n_estimators=492,
    num_leaves=88,
    learning_rate=0.014666949438705097,
    subsample=0.8766412273868395,
    colsample_bytree=0.5848756135604947,
    random_state=42
)

# ✅ 모델 학습
rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
lgb_model.fit(X_train, y_train)

# ✅ 각 모델의 예측 확률 계산
rf_preds = rf_model.predict_proba(X_val)[:, 1]
xgb_preds = xgb_model.predict_proba(X_val)[:, 1]
lgb_preds = lgb_model.predict_proba(X_val)[:, 1]

# ✅ 가중 평균 앙상블 (가중치 설정)
rf_weight = 0.4
xgb_weight = 0.3
lgb_weight = 0.3

final_preds = (rf_weight * rf_preds) + (xgb_weight * xgb_preds) + (lgb_weight * lgb_preds)
final_preds_binary = (final_preds >= 0.5).astype(int)  # 0.5 이상이면 1, 아니면 0


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031973 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 720
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 62
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
🔥 Confusion Matrix 🔥
[[36974  1051]
 [12000  1246]]

🔥 Classification Report 🔥
              precision    recall  f1-score   support

           0       0.75      0.97      0.85     38025
           1       0.54      0.09      0.16     13246

    accuracy                           0.75     51271
   macro avg       0.65      0.53      0.51     51271
weighted a

In [83]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

# ✅ 타겟 변수와 특성 분리
y = X_train_encoded['임신 성공 여부']
X = X_train_encoded.drop('임신 성공 여부', axis=1)

# ✅ 데이터 분할 (훈련 / 검증)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [45]:
# ✅ 평가 지표 확인
conf_matrix = confusion_matrix(y_val, final_preds_binary).astype(float)

print("🔥 Confusion Matrix 🔥")
print(np.round(conf_matrix, 4))  # 소수점 4자리까지 반올림하여 출력

print("\n🔥 Classification Report 🔥")
print(classification_report(y_val, final_preds_binary, digits=4))  # classification report도 소수점 4자리로 출력

🔥 Confusion Matrix 🔥
[[37000.  1025.]
 [12015.  1231.]]

🔥 Classification Report 🔥
              precision    recall  f1-score   support

           0     0.7549    0.9730    0.8502     38025
           1     0.5457    0.0929    0.1588     13246

    accuracy                         0.7457     51271
   macro avg     0.6503    0.5330    0.5045     51271
weighted avg     0.7008    0.7457    0.6716     51271



In [85]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# ✅ IVF(1) 데이터만 선택
ivf_mask = (X_train["시술 유형"] == 1)
X_train_ivf = X_train[ivf_mask]
y_train_ivf = y_train[ivf_mask]

# ✅ IVF(1) 데이터로만 train-validation split
X_train_ivf_split, X_val_ivf_split, y_train_ivf_split, y_val_ivf_split = train_test_split(
    X_train_ivf, y_train_ivf, test_size=0.2, random_state=42
)

# ✅ RandomForest 모델 학습
rf_model = RandomForestClassifier(
    n_estimators=300, 
    max_depth=15, 
    min_samples_split=5, 
    class_weight="balanced",  # 클래스 불균형 해결
    random_state=42
)
rf_model.fit(X_train_ivf_split, y_train_ivf_split)

# ✅ XGBoost 모델 학습 (scale_pos_weight 적용)
xgb_model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=3,  # 양성(1) 가중치 증가 (불균형 데이터 해결)
    random_state=42
)
xgb_model.fit(X_train_ivf_split, y_train_ivf_split)

# ✅ LightGBM 모델 학습 (scale_pos_weight 적용)
lgb_model = LGBMClassifier(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=10,
    num_leaves=50,
    subsample=0.9,
    colsample_bytree=0.9,
    scale_pos_weight=3,  # 양성(1) 데이터 가중치 증가
    min_child_samples=20,
    max_bin=300,  # 이진화 개수 증가
    random_state=42
)
lgb_model.fit(X_train_ivf_split, y_train_ivf_split)

# ✅ 전체 검증 데이터에 대한 예측 수행
rf_preds = rf_model.predict_proba(X_val)[:, 1]
xgb_preds = xgb_model.predict_proba(X_val)[:, 1]
lgb_preds = lgb_model.predict_proba(X_val)[:, 1]

# ✅ 가중 평균 앙상블
rf_weight, xgb_weight, lgb_weight = 0.3, 0.4, 0.3
final_preds = (rf_weight * rf_preds) + (xgb_weight * xgb_preds) + (lgb_weight * lgb_preds)

# ✅ DI(0)인 경우 강제로 0으로 예측
final_preds_binary = (final_preds >= 0.5).astype(int)
final_preds_binary[X_val["시술 유형"] == 0] = 0

# ✅ Confusion Matrix 원본 출력
cm = confusion_matrix(y_val, final_preds_binary)
print("\n🔥 Overall Confusion Matrix 🔥")
print(cm)

# ✅ Classification Report (소수점 4째자리까지)
print("\n🔥 Overall Classification Report 🔥")
print(classification_report(y_val, final_preds_binary, digits=4))

submission_path = '/Users/yudayeon/Desktop/LGAimers/submission/sample_submission.csv'
sample_submission = pd.read_csv(submission_path)
sample_submission['probability'] = final_preds

# CSV 파일로 저장
output_path = '/Users/yudayeon/Desktop/LGAimers/submission/baseline_submit2.csv'
sample_submission.to_csv(output_path, index=False)

print(f"예측 결과가 '{output_path}'에 저장되었습니다.")

KeyboardInterrupt: 

In [77]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# ✅ IVF(1) 데이터만 선택
ivf_mask = (X_train["시술 유형"] == 1)
X_train_ivf = X_train[ivf_mask]
y_train_ivf = y_train[ivf_mask]

# ✅ IVF(1) 데이터로만 train-validation split
X_train_ivf_split, X_val_ivf_split, y_train_ivf_split, y_val_ivf_split = train_test_split(
    X_train_ivf, y_train_ivf, test_size=0.2, random_state=42
)

# ✅ RandomForest 모델 학습
rf_model = RandomForestClassifier(
    n_estimators=300, 
    max_depth=15, 
    min_samples_split=5, 
    class_weight="balanced",  # 클래스 불균형 해결
    random_state=42
)
rf_model.fit(X_train_ivf_split, y_train_ivf_split)

# ✅ XGBoost 모델 학습 (scale_pos_weight 적용)
xgb_model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=3,  # 양성(1) 가중치 증가 (불균형 데이터 해결)
    random_state=42
)
xgb_model.fit(X_train_ivf_split, y_train_ivf_split)

# ✅ LightGBM 모델 학습 (scale_pos_weight 적용)
lgb_model = LGBMClassifier(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=10,
    num_leaves=50,
    subsample=0.9,
    colsample_bytree=0.9,
    scale_pos_weight=3,  # 양성(1) 데이터 가중치 증가
    min_child_samples=20,
    max_bin=300,  # 이진화 개수 증가
    random_state=42
)
lgb_model.fit(X_train_ivf_split, y_train_ivf_split)

# ✅ 전체 검증 데이터에 대한 예측 수행
rf_preds = rf_model.predict_proba(X_val)[:, 1]
xgb_preds = xgb_model.predict_proba(X_val)[:, 1]
lgb_preds = lgb_model.predict_proba(X_val)[:, 1]

# ✅ 가중 평균 앙상블
rf_weight, xgb_weight, lgb_weight = 0.4, 0.3, 0.3
final_preds = (rf_weight * rf_preds) + (xgb_weight * xgb_preds) + (lgb_weight * lgb_preds)

# ✅ DI(0)인 경우 강제로 0으로 예측
final_preds_binary = (final_preds >= 0.5).astype(int)
final_preds_binary[X_val["시술 유형"] == 0] = 0

# ✅ Confusion Matrix 원본 출력
cm = confusion_matrix(y_val, final_preds_binary)
print("\n🔥 Overall Confusion Matrix 🔥")
print(cm)

# ✅ Classification Report (소수점 4째자리까지)
print("\n🔥 Overall Classification Report 🔥")
print(classification_report(y_val, final_preds_binary, digits=4))

# ✅ 제출 파일 로드
submission_path = '/Users/yudayeon/Desktop/LGAimers/submission/sample_submission.csv'
sample_submission = pd.read_csv(submission_path)

# ✅ 테스트 데이터에 대한 예측 수행
rf_preds_test = rf_model.predict_proba(X_test)[:, 1]
xgb_preds_test = xgb_model.predict_proba(X_test)[:, 1]
lgb_preds_test = lgb_model.predict_proba(X_test)[:, 1]

# ✅ 가중 평균 앙상블
final_preds_test = (rf_weight * rf_preds_test) + (xgb_weight * xgb_preds_test) + (lgb_weight * lgb_preds_test)

# ✅ DI(0)인 경우 강제로 0으로 예측
final_preds_test_binary = (final_preds_test >= 0.5).astype(int)
final_preds_test_binary[X_test["시술 유형"] == 0] = 0

# ✅ 제출 파일 생성
sample_submission['probability'] = final_preds_test

# ✅ CSV 파일로 저장
output_path = '/Users/yudayeon/Desktop/LGAimers/submission/baseline_submit3.csv'
sample_submission.to_csv(output_path, index=False)

print(f"예측 결과가 '{output_path}'에 저장되었습니다.")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 41871, number of negative: 118160
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021062 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 691
[LightGBM] [Info] Number of data points in the train set: 160031, number of used features: 61
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.261643 -> initscore=-1.037446
[LightGBM] [Info] Start training from score -1.037446
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

🔥 Overall Confusion Matrix 🔥
[[21965 16060]
 [ 3159 10087]]

🔥 Overall Classification Report 🔥
              precision    recall  f1-score   support

           0     0.8743    0.5776    0.6957     38025
           1     0.3858    0.7615    0.5121     13246

    accuracy              

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- ID
- probability
Feature names seen at fit time, yet now missing:
- DI 시술 횟수
- DI 임신 횟수
- DI 출산 횟수
- IVF 시술 횟수
- IVF 임신 횟수
- ...


In [97]:
# ✅ IVF(1) 데이터만 선택
ivf_mask = (X_train["시술 유형"] == 1)
X_train_ivf = X_train[ivf_mask]
y_train_ivf = y_train[ivf_mask]

# ✅ IVF(1) 데이터로만 train-validation split
X_train_ivf_split, X_val_ivf_split, y_train_ivf_split, y_val_ivf_split = train_test_split(
    X_train_ivf, y_train_ivf, test_size=0.2, random_state=42
)

# ✅ RandomForest 모델 학습
rf_model = RandomForestClassifier(
    n_estimators=300, 
    max_depth=15, 
    min_samples_split=5, 
    class_weight="balanced",  # 클래스 불균형 해결
    random_state=42
)
rf_model.fit(X_train_ivf_split, y_train_ivf_split)

# ✅ XGBoost 모델 학습 (scale_pos_weight 적용)
xgb_model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=10,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=3,  # 양성(1) 가중치 증가 (불균형 데이터 해결)
    random_state=42
)
xgb_model.fit(X_train_ivf_split, y_train_ivf_split)

# ✅ LightGBM 모델 학습 (scale_pos_weight 적용)
lgb_model = LGBMClassifier(
    n_estimators=300,
    learning_rate=0.03,
    max_depth=10,
    num_leaves=50,
    subsample=0.9,
    colsample_bytree=0.9,
    scale_pos_weight=3,  # 양성(1) 데이터 가중치 증가
    min_child_samples=20,
    max_bin=300,  # 이진화 개수 증가
    random_state=42
)
lgb_model.fit(X_train_ivf_split, y_train_ivf_split)

# ✅ 테스트 데이터 로드
test_data_path = '/Users/yudayeon/Desktop/LGAimers/data/test_encoded.csv'
X_test = pd.read_csv(test_data_path)

# ✅ X_test에서 X_train과 동일한 컬럼만 선택
X_test = X_test[X_train.columns]

# ✅ 테스트 데이터에 대한 예측 수행
rf_preds_test = rf_model.predict_proba(X_test)[:, 1]
xgb_preds_test = xgb_model.predict_proba(X_test)[:, 1]
lgb_preds_test = lgb_model.predict_proba(X_test)[:, 1]

# ✅ 가중 평균 앙상블
final_preds_test = (rf_weight * rf_preds_test) + (xgb_weight * xgb_preds_test) + (lgb_weight * lgb_preds_test)

# ✅ DI(0)인 경우 강제로 0으로 예측
final_preds_test_binary = (final_preds_test >= 0.5).astype(int)
final_preds_test_binary[X_test["시술 유형"] == 0] = 0

# ✅ Confusion Matrix 원본 출력
cm = confusion_matrix(y_val, final_preds_binary)
print("\n🔥 Overall Confusion Matrix 🔥")
print(cm)

# ✅ Classification Report (소수점 4째자리까지)
print("\n🔥 Overall Classification Report 🔥")
print(classification_report(y_val, final_preds_binary, digits=4))

# ✅ 제출 파일 로드
submission_path = '/Users/yudayeon/Desktop/LGAimers/submission/sample_submission.csv'
sample_submission = pd.read_csv(submission_path)

# ✅ 예측값 저장
sample_submission['probability'] = final_preds_test

# ✅ CSV 파일로 저장
output_path = '/Users/yudayeon/Desktop/LGAimers/submission/baseline_submit3.csv'
sample_submission.to_csv(output_path, index=False)

print(f"예측 결과가 '{output_path}'에 저장되었습니다.")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 41871, number of negative: 118160
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018322 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 691
[LightGBM] [Info] Number of data points in the train set: 160031, number of used features: 61
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.261643 -> initscore=-1.037446
[LightGBM] [Info] Start training from score -1.037446
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

🔥 Overall Confusion Matrix 🔥
[[21965 16060]
 [ 3159 10087]]

🔥 Overall Classification Report 🔥
              precision    recall  f1-score   support

           0     0.8743    0.5776    0.6957     38025
           1     0.3858    0.7615    0.5121     13246

    accuracy              